In [3]:
import os

# ===== CONFIG =====
BASE_FOLDER = "digitDataset"  # parent folder containing 0, 1, 2, ...
EXTENSION = ".jpg"
START_NUMBER = 191
END_NUMBER = 200  # maximum number per folder
# ===================

for folder_name in sorted(os.listdir(BASE_FOLDER)):
    folder_path = os.path.join(BASE_FOLDER, folder_name)
    if not os.path.isdir(folder_path):
        continue

    files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    
    for idx, filename in enumerate(files):
        old_path = os.path.join(folder_path, filename)
        new_number = START_NUMBER + idx

        if new_number > END_NUMBER:
            # Delete extra files
            os.remove(old_path)
            print(f"🗑 Deleted extra file: {old_path}")
            continue

        new_name = f"{folder_name}{new_number}{EXTENSION}"
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed {old_path} → {new_path}")

print("✅ All folders processed successfully!")


Renamed digitDataset\0\0191.jpg → digitDataset\0\0191.jpg
Renamed digitDataset\0\0192.jpg → digitDataset\0\0192.jpg
Renamed digitDataset\0\0193.jpg → digitDataset\0\0193.jpg
Renamed digitDataset\0\0194.jpg → digitDataset\0\0194.jpg
Renamed digitDataset\0\0195.jpg → digitDataset\0\0195.jpg
Renamed digitDataset\0\0196.jpg → digitDataset\0\0196.jpg
Renamed digitDataset\0\0197.jpg → digitDataset\0\0197.jpg
Renamed digitDataset\0\0198.jpg → digitDataset\0\0198.jpg
Renamed digitDataset\0\0199.jpg → digitDataset\0\0199.jpg
Renamed digitDataset\0\0200.jpg → digitDataset\0\0200.jpg
🗑 Deleted extra file: digitDataset\0\0201.jpg
🗑 Deleted extra file: digitDataset\0\0202.jpg
🗑 Deleted extra file: digitDataset\0\0203.jpg
🗑 Deleted extra file: digitDataset\0\0204.jpg
🗑 Deleted extra file: digitDataset\0\0205.jpg
🗑 Deleted extra file: digitDataset\0\0206.jpg
🗑 Deleted extra file: digitDataset\0\0207.jpg
🗑 Deleted extra file: digitDataset\0\0208.jpg
Renamed digitDataset\1\1191.jpg → digitDataset\1\119

In [12]:

import pyrealsense2 as rs
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


# Create 1000 random 3D points
depth_points = np.random.rand(1000, 3)  # Nx3 array
color_points = np.random.rand(1000, 3)  # Nx3 array, RGB values 0-1

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(depth_points)
pcd.colors = o3d.utility.Vector3dVector(color_points)

o3d.visualization.draw_geometries([pcd])


ModuleNotFoundError: No module named 'pyrealsense2'

In [13]:
import pyrealsense2 as rs
import numpy as np
import open3d as o3d

# -------------------------------
# Configure RealSense pipeline
# -------------------------------
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

pipeline.start(config)

# Create Open3D visualizer
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Live 3D Point Cloud')

point_cloud = o3d.geometry.PointCloud()
geom_added = False

try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Create point cloud
        intrinsics = depth_frame.profile.as_video_stream_profile().intrinsics
        fx, fy = intrinsics.fx, intrinsics.fy
        cx, cy = intrinsics.ppx, intrinsics.ppy

        height, width = depth_image.shape
        xs, ys = np.meshgrid(np.arange(width), np.arange(height))
        zs = depth_image / 1000.0  # Convert mm to meters
        xs = (xs - cx) * zs / fx
        ys = (ys - cy) * zs / fy

        points = np.stack((xs, ys, zs), axis=-1).reshape(-1, 3)
        colors = color_image.reshape(-1, 3) / 255.0

        point_cloud.points = o3d.utility.Vector3dVector(points)
        point_cloud.colors = o3d.utility.Vector3dVector(colors)

        if not geom_added:
            vis.add_geometry(point_cloud)
            geom_added = True

        vis.update_geometry(point_cloud)
        vis.poll_events()
        vis.update_renderer()

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    pipeline.stop()
    vis.destroy_window()


RuntimeError: No device connected